In [1]:
import os
import sys
import git
import pathlib

import random

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ['CUDA_VISIBLE_DEVICES'] = '0' # use GPU
# Using GPU during inference has deterministic results (same as CPU)

PROJ_ROOT_PATH = pathlib.Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
PROJ_ROOT =  str(PROJ_ROOT_PATH)
if PROJ_ROOT not in sys.path:
    sys.path.append(PROJ_ROOT)

from libs import utils, mnist32_cnn
from libs.constants import model_seeds

In [2]:
# Limit GPU growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [3]:
# Prepare dataset
# Combine test and train images together into one dataset
DATASET_PATH = str(pathlib.Path(PROJ_ROOT_PATH / "datasets" / "mnist.npz" ))
(train_images, train_labels), (test_images, test_labels) = mnist.load_data(path=DATASET_PATH)
train_images = train_images.astype(np.float32) / 255.0
test_images = test_images.astype(np.float32) / 255.0  

all_images =np.concatenate([train_images, test_images], axis=0)
all_labels =np.concatenate([train_labels, test_labels], axis=0)
all_images = np.expand_dims(all_images, axis=-1)

# resize the input shape , i.e. old shape: 28, new shape: 32
image_x_size = 32
image_y_size = 32
all_images = tf.image.resize(all_images, [image_x_size, image_y_size]) 

In [4]:
model_type = "mnist32-cnn_1024_256_64"
model_seed = model_seeds[0]

In [5]:
# Get model
model_instance = model_type + "-" + str(model_seed)
dataset, model_arch, model_config, layer_widths, seed = utils.instancename2metadata(model_instance)
model_meta_type, model_type, model_instance = utils.metadata2instancenames(dataset, model_arch, layer_widths, seed)

model_folder = pathlib.Path(PROJ_ROOT_PATH / "models" / model_meta_type / model_type)
model_filename = model_instance + ".h5"
model_file = pathlib.Path(model_folder/ model_filename)

In [6]:
no_im = 32 # no of images per batch
b_images = all_images[0:no_im]
b_labels = all_labels[0:no_im]
# image = tf.expand_dims(all_images[0],axis=0)

In [7]:
# Load model
model = tf.keras.models.load_model(model_file)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 29, 29, 32)        544       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 fc_0 (Dense)                (None, 1024)              6423552   
                                                                 
 fc_1 (Dense)                (None, 256)               262400    
                                                                 
 fc_2 (Dense)                (None, 64)                16448     
                                                        

In [8]:
from libs.fitnessfns import batch_lenet_3hidden_ERRexpbitflips

In [9]:
misses = batch_lenet_3hidden_ERRexpbitflips( b_images,
                                    b_labels,
                                    model,
                                    error_profile_c0=None,
                                    error_profile_h0=None,
                                    error_profile_h1=None,
                                    error_profile_h2=None,
                                    error_profile_op=None,
                                    ERR_PARAM_TF=None,
                                    clayer0_shuffle_order=None,
                                    hlayer0_shuffle_order=None,
                                    hlayer1_shuffle_order=None,
                                    hlayer2_shuffle_order=None,
                                    oplayer_shuffle_order=None)

In [10]:
misses

<tf.Tensor: shape=(), dtype=int64, numpy=0>

In [11]:
dummy_error_profile = np.zeros((20_000,32), dtype="float32")
clayer0_shuffle_order = np.arange(32)
hlayer0_shuffle_order = np.arange(1024)
hlayer1_shuffle_order = np.arange(256)
hlayer2_shuffle_order = np.arange(64)
oplayer_shuffle_order = np.arange(10)

np.random.shuffle(clayer0_shuffle_order)
np.random.shuffle(hlayer0_shuffle_order)
np.random.shuffle(hlayer1_shuffle_order)
np.random.shuffle(hlayer2_shuffle_order)
np.random.shuffle(oplayer_shuffle_order)

In [12]:
misses = batch_lenet_3hidden_ERRexpbitflips( b_images,
                                            b_labels,
                                            model,
                                            error_profile_c0=dummy_error_profile,
                                            error_profile_h0=dummy_error_profile,
                                            error_profile_h1=dummy_error_profile,
                                            error_profile_h2=dummy_error_profile,
                                            error_profile_op=dummy_error_profile,
                                            ERR_PARAM_TF=None,
                                            clayer0_shuffle_order=clayer0_shuffle_order,
                                            hlayer0_shuffle_order=hlayer0_shuffle_order,
                                            hlayer1_shuffle_order=hlayer1_shuffle_order,
                                            hlayer2_shuffle_order=hlayer2_shuffle_order,
                                            oplayer_shuffle_order=oplayer_shuffle_order
                                                   )

In [13]:
from libs.fitnessfns import ff_lenet_3hidden_ERRexpbitflips

In [15]:
test_set = (all_images, all_labels)
accuracy = ff_lenet_3hidden_ERRexpbitflips(model,
                                            error_profile_c0=dummy_error_profile,
                                            error_profile_h0=dummy_error_profile,
                                            error_profile_h1=dummy_error_profile,
                                            error_profile_h2=dummy_error_profile,
                                            error_profile_op=dummy_error_profile,
                                            ERR_PARAM=None,
                                            clayer0_shuffle_order=clayer0_shuffle_order,
                                            hlayer0_shuffle_order=hlayer0_shuffle_order,
                                            hlayer1_shuffle_order=hlayer1_shuffle_order,
                                            hlayer2_shuffle_order=hlayer2_shuffle_order,
                                            oplayer_shuffle_order=oplayer_shuffle_order,
                                            test_set=test_set,
                                            batchsize=32)

In [16]:
accuracy

<tf.Tensor: shape=(), dtype=float64, numpy=0.9957990397805213>

In [18]:
from libs.mnist32_cnn import test_mnist32
acc2, _ = test_mnist32(model_file)

In [19]:
acc2

0.9958

In [20]:
accuracy-acc2

<tf.Tensor: shape=(), dtype=float64, numpy=-9.602194787072804e-07>